<a href="https://colab.research.google.com/github/ukhyun-lee/stock_market_analysis/blob/main/4%EC%A3%BC%EC%B0%A8/wordcloud2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경설정

In [ ]:
# 한국어 텍스트 전처리를 위한 konlpy 패키지 설치
!pip install konlpy

In [ ]:
# 한글 폰트(나눔 폰트) 설치
!sudo apt-get install -y fonts-nanum

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime

title_list = []
url_list = []
article_list = []
date_list = []

for i in range(1):
    date= datetime.date.today() - datetime.timedelta(days = i)
    url = f'https://finance.naver.com/news/mainnews.naver?date={date}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')

    titles = soup.find_all('dd', {'class': 'articleSubject'})
    for t in titles:  
        title = t.text.strip()
        url = t.find('a').get('href')
        response = requests.get('https://finance.naver.com/' + url) # 상세 기사 페이지
        soup = BeautifulSoup(response.text, 'lxml')
        article = soup.find('div', {'class': 'articleCont'})
        
        title_list.append(title)
        url_list.append(url)
        if article is not None:
            article_list.append(article.text.strip())
        else:
            article_list.append('')
        date_list.append(date)

df = pd.DataFrame({'기사제목': title_list, '본문url': url_list, '기사본문': article_list, '날짜': date_list})
df.to_csv('article.csv', encoding = 'euc-kr')
     

In [ ]:
# 워드 클라우드를 만들기 위한 작업
import matplotlib.pyplot as plt
from collections import Counter
from konlpy.tag import Okt
# article = '''
# 20일 아시아 주요 증시는 보합권에서 등락이 엇갈렸다.

# 이날 일본 도쿄의 닛케이225지수는 전날보다 0.18% 오른 2만8657.57에 거래를 마쳤다.

# 일본 증시는 이날 뚜렷한 방향을 찾지 못한 채 보합권에서 상승과 하락을 반복하다가 정부의 정책 기대 속에 반도체주 상승에 힘입어 오름세로 마감했다고 니혼게이자이는 전했다.

# 매체에 따르면 기시다 후미오 일본 총리는 외국인투자 촉진을 위해 반도체 등 분야에서 공급망 구축의 수치 목표를 담은 액션 플랜을 이달 안에 발표할 예정으로 알려졌다. 이에 일본의 대표적인 반도체 장비업체 도쿄일렉트론이 1.75% 오르는 등 반도체 업종이 강세를 보이며 지수를 밀어올렸다.

# 중화권에선 본토 상하이종합지수가 전일 대비 0.09% 하락한 3367.03에, 대만 자취안지수가 0.62% 내린 1만5770.47에 각각 마감했다. 반면 홍콩 항셍지수는 0.14% 오른 2만396.97에 장을 종료했다.

# 한편 이날 중국 인민은행은 사실상 기준금리인 대출우대금리(LPR)를 동결했다. 지난해 8월 5bp(1bp=0.01%포인트) 내린 이후 8개월 연속 동결이다. 예상됐던 결과로 시장에 별다른 영향은 없었다.'''

article = df['기사본문'][0]
article

In [ ]:
# 기사의 내용을 토큰으로 만들기
okt = Okt()
tokens = okt.nouns(article)
counted_tokens = Counter(tokens)
top_20 = counted_tokens.most_common(20)
print(top_20)

In [ ]:
from wordcloud import WordCloud
wc = WordCloud(background_color='white', font_path='NanumBarunGothic.ttf')
wc.generate_from_frequencies(counted_tokens)
figure = plt.figure()
figure.set_size_inches(10, 10)
ax = figure.add_subplot(1, 1, 1)
ax.axis("off")
ax.imshow(wc)